In [160]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import sklearn.cross_validation
from sklearn.cross_validation import cross_val_score

In [161]:
def setPred(col):
    if(col > 0):
        return 1
    else:
        return 0
    
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

# preparar/tweks datos 

In [162]:
features = pd.read_csv("features.csv",low_memory = False)
prediccion = pd.read_csv("csv/labels_training_set.csv")

In [163]:
#mergeo features y la info q nos tiran, lleno con 0 los q no se
train = pd.merge(prediccion,features,on="person",how= "left").dropna()

In [164]:
prediccion = train["label"]

In [165]:
test = features.loc[~features["person"].isin(train["person"])]

In [166]:
listaDeCategoricos = ["condition_no_convercion","model_no_convercion","color_no_convercion","search_engine"\
                      ,"nombre mas usado dia","ultimo_dia_de_la_semana_activo","ultimo_evento_registrado_cmode"]
for word in listaDeCategoricos:
    train[word],test[word] =  target_encode(trn_series = train[word],  \
                                            tst_series = test[word],\
                                            target=prediccion, \
                                            min_samples_leaf=50,\
                                            smoothing=5,\
                                            noise_level=0.01)


/home/tito/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [167]:
train.columns

Index(['person', 'label', 'ad campaign hit', 'brand listing', 'checkout',
       'conversion', 'generic listing', 'lead', 'search engine hit',
       'searched products', 'staticpage', 'viewed product', 'visited site',
       'model_no_convercion', 'storage_no_convercion',
       'condition_no_convercion', 'color_no_convercion', 'search_engine',
       'numero mas usado dia', 'nombre mas usado dia',
       'ultimo_dia_de_la_semana_activo', 'ultimo_evento_registrado_cmode',
       'dias_hasta', 'viewed product_last', 'searched products_last',
       'ad campaign hit_last', 'staticpage_last', 'checkout_last',
       'search engine hit_last', 'conversion_last', 'generic listing_last',
       'brand listing_last', 'visited site_last', 'lead_last',
       'cantidad_de_dias_utilizado'],
      dtype='object')

In [168]:
exclude = ["ultimo_dia_de_la_semana_activo","cantidad_de_dias_utilizado"]
train = train.drop(exclude,axis = 1)
test = test.drop(exclude,axis = 1)

In [169]:
train = train.drop(["person","label"],axis = 1)

In [170]:
for word in features.columns:
    features[word] = pd.to_numeric(features[word],errors = "coerce")

In [171]:

X_train, X_test, y_train, y_test = train_test_split(train, prediccion, test_size=0.30, random_state=42)

# Random Forest
# historial cambios tweaks | score

columnas: 
'ad campaign hit', 'brand listing', 'checkout', 'generic listing',
       'lead', 'search engine hit', 'searched products', 'staticpage',
       'viewed product', 'visited site', 'level_0', 'storage_no_convercion',
       'numero mas usado dia', 'condition_no_convercion2',
       'color_no_convercion2', 'search_engine2', 'nombre mas usado dia2']
       
features contraproductivos:
    cantidad de dias utilizado, hora/mes/dia ultimo evento, ultimo dia de la semana activo

test_size = 0.3
cv = 20
n_estimator 50 | .78 (Kaggle .76)
n_estimator 100 | .79 

cambio smootingbasado en: conversiones previas -> conversiones futuras

test_size = 0.3
cv = 20
n_estimator 100 | .80 (Kaggle .78)

agrego ultimo evento

test_size = 0.3
cv = 20
n_estimator 100 | .793


agrego: ultimo dias hasta el 1/6,suceso de cada evento, rellenados con -1

test_size = 0.3
cv = 20
n_estimator 100 | .798

agrego: ultimo dia de la semana activo

test_size = 0.3
cv = 20
n_estimator 100 | .798




https://www.youtube.com/watch?v=BSUMBBFjxrY

In [172]:
# probar clasificacion
randomforesttree = RandomForestRegressor(random_state = None, n_jobs=-1)
randomforesttree.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [173]:
#
param_grid= {'max_features': ["auto"], 'n_estimators': [100], 'criterion': ['mse']}
grid_drop = GridSearchCV(randomforesttree, param_grid, cv=20, scoring='roc_auc',n_jobs = -1,pre_dispatch = 4)
grid_drop.fit(X_train, y_train)
print(grid_drop.best_params_, grid_drop.best_score_)

{'max_features': 'auto', 'n_estimators': 100, 'criterion': 'mse'} 0.7986491667875926


In [174]:
result = grid_drop.best_estimator_.predict(X_test)

In [175]:
roc_auc_score(y_test, result)

0.8022901093439969

In [132]:
feature_importances = pd.DataFrame(grid_drop.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [133]:
feature_importances

,importance
color_no_convercion,0.093372
search_engine,0.088687
nombre mas usado dia,0.088457
model_no_convercion,0.086945
condition_no_convercion,0.086425
ultimo_evento_registrado_cmode,0.083037
checkout,0.079305
viewed product,0.056703
numero mas usado dia,0.055616
brand listing,0.045071


# Gradient Boost
# historial cambios tweaks | score 

columnas: 
'ad campaign hit', 'brand listing', 'checkout', 'generic listing',
       'lead', 'search engine hit', 'searched products', 'staticpage',
       'viewed product', 'visited site', 'level_0', 'storage_no_convercion',
       'numero mas usado dia', 'condition_no_convercion2',
       'color_no_convercion2', 'search_engine2', 'nombre mas usado dia2']


cambio smootingbasado en: conversiones previas -> conversiones futuras

agrego ultimo evento

agrego dias hasta el 1/6

test_size = 0.3
cv = 20
n_estimator 100 | .82 (Kaggle .815)


agrego: ultimo dia de semana

test_size = 0.3
cv = 20
n_estimator 100 | .820

agrego: ultimo suceso de cada evento, rellenados con -1

test_size = 0.3
cv = 20
n_estimator 100 | .824

agrego: cantidad de dias utilizado

test_size = 0.3
cv = 20
n_estimator 100 | .824



In [77]:
gradientboost = GradientBoostingRegressor(random_state = None)
gradientboost.get_params()

{'alpha': 0.9,
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'presort': 'auto',
 'random_state': None,
 'subsample': 1.0,
 'verbose': 0,
 'warm_start': False}

In [95]:
param_grid= {'max_features': ["auto"], 'n_estimators': [30], 'loss': ['ls']}
grid_drop = GridSearchCV(gradientboost, param_grid, cv=20, scoring='roc_auc')
grid_drop.fit(X_train, y_train)
print(grid_drop.best_params_, grid_drop.best_score_)

{'max_features': 'auto', 'loss': 'ls', 'n_estimators': 30} 0.8248068435689075


In [96]:
result2 = grid_drop.best_estimator_.predict(X_test)

In [97]:
roc_auc_score(y_test, result2)

0.8327988141103331

In [214]:
feature_importances2 = pd.DataFrame(grid_drop.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
checkout,0.082237
nombre mas usado dia,0.076558
ultimo_evento_registrado_cmode,0.075809
color_no_convercion,0.075315
ultimo_dia_de_la_semana_activo,0.072172
search_engine,0.070993
condition_no_convercion,0.070082
model_no_convercion,0.062863
viewed product,0.048716
numero mas usado dia,0.044006


# Summit



In [40]:
to_pred = pd.read_csv("csv/trocafone_kaggle_test.csv")
features = pd.read_csv("features0,1.csv")

In [37]:
to_pred_completo = pd.merge(to_pred,test,on="person",how = "left").dropna()

In [38]:
to_pred_completo["label"] = grid_drop.best_estimator_.predict(to_pred_completo.drop(["person"],axis = 1))

ValueError: Number of features of the model must  match the input. Model n_features is 34 and  input n_features is 31 

In [39]:
to_pred = pd.merge(to_pred_completo[["person","label"]],to_pred,on="person",how="right")

KeyError: "['label'] not in index"

In [198]:
to_pred = to_pred.fillna(to_pred["label"].median())

In [199]:
to_pred.to_csv("sumit0.0", index=False)

In [34]:
to_pred.head()

NameError: name 'to_pred' is not defined

5